###### The current process is to copy the sas data set to jupyter folder to process, 

* to prevent from using the file while the file is being written
* need to process to copy the file directly from project folder to juypter folder

In [14]:
# import libraries
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 100)
import datetime

In [2]:
df = pd.read_sas('patpop_ps.sas7bdat')

In [8]:
df.dtypes

patientid                               float64
sex                                     float64
age                                     float64
index_dt                         datetime64[ns]
cohort                                  float64
Obesity_dx                              float64
Overweight                              float64
Smoking                                 float64
Alcohol_abuse                           float64
Drug_abuse                              float64
Obesity_rx                              float64
BMI                                     float64
Obesity                                 float64
Diabetic_retinopathy                    float64
Diabetes_WOOM                           float64
RD_VH_vitrectomy                        float64
Diabetic_neuropathy                     float64
Diabetic_nephropathy                    float64
Hypoglycemia_p1                         float64
Hyperglycemia                           float64
Disorders_FE_ADD                        

In [172]:
# create age group
def age_group(age):
    if age >= 18 and age <= 54:
        return '18-54'
    elif age >= 55 and age <= 64:
        return '55-64'
    elif age >= 65 and age <= 74:
        return '65-74'
    else:
        return '75+'

def sex(x):
    if x == 1:
        return 'male'
    if x == 2:
        return 'female'

In [173]:
df['age_group'] = df['age'].apply(lambda x: age_group(x))
df['sex_name'] = df['sex'].apply(lambda x: sex(x))

In [171]:
# create contigency tables
def cross(ind, col):
    return pd.crosstab(index=ind, columns=df[col])

In [174]:
cohort_tab = cross('number of patients', 'cohort')
age_tab = cross(df['age_group'], 'cohort')
sex_tab = cross(df['sex_name'], 'cohort')

In [166]:
def numeric_tab():
    df.age = df.age.astype(int)
    age_mea = df.groupby(['cohort']).age.mean().round(2).to_frame().rename(columns={'age':'age_mean'}).transpose()
    age_s = df.groupby(['cohort']).age.std().round(2).to_frame().rename(columns={'age':'age_std'}).transpose()
    age_med = df.groupby(['cohort']).age.median().round(2).to_frame().rename(columns={'age':'age_median'}).transpose()
    age_q25 = df.groupby(['cohort']).age.quantile(0.25).round(2).to_frame()
    age_q75 = df.groupby(['cohort']).age.quantile(0.75).round(2).to_frame()
    age_min = df.groupby(['cohort']).age.min().round(2).to_frame()
    age_max = df.groupby(['cohort']).age.max().round(2).to_frame()

    age_q = age_q25.rename(columns={'age':'age_Q1'}).join(age_q75.rename(columns={'age': 'age_Q3'}))
    age_q['age_Q1_Q3'] = age_q['age_Q1'].map(str) + '; ' + age_q['age_Q3'].map(str)
    age_q = age_q.drop(['age_Q1', 'age_Q3'], axis=1).transpose()

    age_m = age_min.rename(columns={'age':'age_min'}).join(age_max.rename(columns={'age': 'age_max'}))
    age_m['age_min_max'] = age_m['age_min'].map(str) + '; ' + age_m['age_max'].map(str)
    age_m = age_m.drop(['age_min', 'age_max'], axis=1).transpose()

    age_std_data = {'std': np.absolute(age_mea.iloc[0,0] - age_mea.iloc[0,1])/np.sqrt((np.square(age_s.iloc[0,0]) + np.square(age_s.iloc[0,1]))/2)}
    age_std = pd.DataFrame(age_std_data, index=['age_mean'])

    age_df = pd.concat([age_mea, age_s, age_med, age_q, age_m])
    age_all = age_df.join(age_std.round(3))

    age_all.fillna('NA')

,1.0,2.0,std
age_mean,59.17,59.12,0.004
age_std,13.83,14.12,NA
age_median,60,60,NA
age_Q1_Q3,49.0; 69.0,49.0; 69.0,NA
age_min_max,18; 90,18; 90,NA


In [158]:
test = np.array([1, 2, 19, 4, 5])
print(type(test))
print(np.quantile(test, 0.5))

<class 'numpy.ndarray'>
4.0


In [175]:
all_tab = pd.concat([cohort_tab, age_df, age_all, sex_tab])
all_tab

,1.0,2.0,std
number of patients,5639,5639,NaN
age_mean,59.17,59.12,NaN
age_std,13.83,14.12,NaN
age_median,60,60,NaN
age_Q1_Q3,49.0; 69.0,49.0; 69.0,NaN
age_min_max,18; 90,18; 90,NaN
age_mean,59.17,59.12,0.004
age_std,13.83,14.12,NaN
age_median,60,60,NaN
age_Q1_Q3,49.0; 69.0,49.0; 69.0,NaN
